# Bibliotecas

Nesta seção vamos instalar todas as bibliotecas utilizadas.

Além do OpenStreetMap, foram utilizadas: geopandas, pandas e networkx, mas estas já estão disponíveis no jupyter.

In [143]:
!pip install osmnx

# Preparação

Nesta seção, vamos preparar o grafo com os dados recolhidos [neste repositório](https://github.com/lemuel-manske/city-block/tree/main/data) e organizar vértices e arestas por bairro.

In [144]:
BLUMENAU = 'Blumenau, Brazil'
TIMBO = 'Timbó, Brazil'
INDAIAL = 'Indaial, Brazil'

In [145]:
from typing import Union

import pandas as pd
import geopandas as gpd

import folium as fl

import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

1. Gerar o grafo composto de Blumenau, Timbó e Indaial (integrantes Lemuel e Lucas são de Blumenau e Rodrigo de Timbó, por isso optamos por manter o vértice em Timbó)

In [146]:
'''
  Get's graph from a given `place`.
'''
def get_graph_from_place(place: str):
  return ox.graph_from_place(
    place,
    network_type='drive'
  )

In [147]:
G = nx.compose_all([
  get_graph_from_place(BLUMENAU),
  get_graph_from_place(TIMBO),
  get_graph_from_place(INDAIAL)
])

2. Importar os dados recolhidos para o grafo. Para este processo, foram coletadas longitude - `x` - e latitude - `y` - de cada ponto de interesse e no grafo vamos atribuir um "label" e "nome" para o vértice mais próximo ao ponto de interesse desejado. Todos os datasets estão disponíveis [neste repositório](http://github.com/lemuel-manske/city-block/data)
  - https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.distance.nearest_nodes

In [459]:
HOSPITAL_TAG = 'hospital'
ESTUDANTE_TAG = 'estudante'
CRECHE_TAG = 'creche'
ESCOLA_TAG = 'escola'
PONTO_SAUDE_TAG = 'ponto_saude'
PSICOSSOCIAL_TAG = 'psicossocial'

BAIRROS_BLUMENAU_TAG = 'bairros_blumenau'
BAIRROS_TIMBO_TAG = 'bairros_timbo'

In [195]:
REPOSITORY = 'https://raw.githubusercontent.com/lemuel-manske/city-block/main/data'

DATA_SETS = {
    HOSPITAL_TAG: REPOSITORY + '/hospitais.csv',
    ESTUDANTE_TAG: REPOSITORY + '/estudantes.csv',
    CRECHE_TAG: REPOSITORY + '/creches.csv',
    ESCOLA_TAG: REPOSITORY + '/escolas.csv',
    PONTO_SAUDE_TAG: REPOSITORY + '/pontos_saude.csv',
    PSICOSSOCIAL_TAG: REPOSITORY + '/centros_psicossociais.csv',
    BAIRROS_BLUMENAU_TAG: REPOSITORY + '/bairros_blumenau.csv',
    BAIRROS_TIMBO_TAG: REPOSITORY + '/bairros_timbo.csv'
}

In [460]:
def load_all_data_sets(G: nx.MultiDiGraph):

  def desserialize(value: str) -> Union[str, bool]:
    if value == 'yes':
      return True

    if value == 'no':
      return False

    return value

  def label_points_of_interest(
      G: nx.MultiDiGraph,
      dataframe: pd.DataFrame,
      label: str, tags:
      list[str]):

    for idx, row in dataframe.iterrows():
      nearest_node_from_interest_point = ox.distance.nearest_nodes(G, X=row['x'], Y=row['y'])

      G.nodes[nearest_node_from_interest_point]['label'] = label

      for tag in tags:
        G.nodes[nearest_node_from_interest_point][tag] = desserialize(row[tag])

  def load_data_set(G: nx.MultiDiGraph, dataset_name: str, tags: list[str]):
    df = pd.read_csv(DATA_SETS[dataset_name])
    label_points_of_interest(G, df, dataset_name, tags)

  load_data_set(G, ESTUDANTE_TAG,    ['ref', 'student_name'])
  load_data_set(G, HOSPITAL_TAG,     ['name', 'is_public'])
  load_data_set(G, CRECHE_TAG,       ['name', 'is_public'])
  load_data_set(G, ESCOLA_TAG,       ['name', 'is_public'])
  load_data_set(G, PONTO_SAUDE_TAG,  ['name', 'is_public'])
  load_data_set(G, PSICOSSOCIAL_TAG, ['name', 'is_public'])

In [461]:
load_all_data_sets(G)

3. Afim de facilitar a análise, vamos separar todas os vértices e arestas por bairro. Foi necessário remover os bairros que não são reconhecidos na divisão administrativa feita pela prefeitura [neste link](https://www.blumenau.sc.gov.br/secretarias/secretaria-de-desenvolvimento-urbano/pagina/historia-sobre-municipio/divisa-administrativa-bairros) mas que o OSM reconhece.

  - https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.features.features_from_place
  - https://wiki.openstreetmap.org/wiki/Key:admin_level

Bairros por nome e população (fonte: Censo 2022)

In [227]:
def get_neighborhoods(place: str, places_to_filter: list[str]):
  return ox.features.features_from_place(place, {'admin_level': '10'}) \
    .query(f'name in {list(places_to_filter)}')

blumenau_neighborhoods = pd.read_csv(DATA_SETS[BAIRROS_BLUMENAU_TAG])
timbo_neighborhoods = pd.read_csv(DATA_SETS[BAIRROS_TIMBO_TAG])

neighborhoods = pd \
  .concat([
    get_neighborhoods(BLUMENAU, blumenau_neighborhoods['name']),
    get_neighborhoods(TIMBO, timbo_neighborhoods['name'])
  ]) \
    .merge(blumenau_neighborhoods[['name', 'hab']], on='name', how='left')

4. Com o geopandas, podemos fazer um [*spatial join*](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.sjoin.html#geopandas.GeoDataFrame.sjoin) dos vértices - `G_nodes` - do grafo original que sobrepoem - `predicate='intersects'` - qualquer polígono - `neighborhoods[['geometry']]`. Por fim, renomeamos a coluna de `name_right` para `name` para manter consistência. A saída são todos os vértices de `G`, porém agora estão identificados por bairro.

 - https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html


In [228]:
def nodes_with_neighborhood():
  nodes, _ = ox.graph_to_gdfs(G)

  return gpd \
    .sjoin(nodes, neighborhoods[['name', 'geometry']], how='left', predicate='intersects') \
    .rename(columns={'name_right': 'name'})['name']

nx.set_node_attributes(G, nodes_with_neighborhood().to_dict(), 'neighborhood_name')

In [462]:
def get_labeled_nodes(label: str):
  nodes, _ = ox.graph_to_gdfs(G)

  return nodes[nodes['label'] == label]

In [463]:
def get_student_node_id(student_name: str):
  students = get_labeled_nodes(ESTUDANTE_TAG)

  student_node = students[students['student_name'] == student_name]

  return student_node.index[0]

In [464]:
def closest_service_from_student(student_name: str, target_service: str) -> list[int]:
  start = get_student_node_id(student_name)

  target_nodes = get_labeled_nodes(target_service)

  curr_path_dist = None
  curr_shortest_path = None

  for _, hospital in target_nodes.iterrows():
    end = hospital.name

    new_path = nx.dijkstra_path(G, start, end, weight='length')

    new_path_dist = nx.dijkstra_path_length(G, start, end)

    if curr_path_dist is None:
      curr_path_dist = new_path_dist
      curr_shortest_path = new_path

    elif new_path_dist < curr_path_dist:
      curr_path_dist = new_path_dist
      curr_shortest_path = new_path

  return curr_shortest_path

In [469]:
G_nodes, G_edges = ox.graph_to_gdfs(G)

G_center = G_nodes.geometry.union_all().centroid

G_nodes_student = get_labeled_nodes(ESTUDANTE_TAG)

G_nodes_hospital = get_labeled_nodes(HOSPITAL_TAG)
G_nodes_daycare = get_labeled_nodes(CRECHE_TAG)
G_nodes_school = get_labeled_nodes(ESCOLA_TAG)
G_nodes_psychiatric_care = get_labeled_nodes(PSICOSSOCIAL_TAG)
G_nodes_health_points = get_labeled_nodes(PONTO_SAUDE_TAG)

# Análise

Nesta seção vamos gerar informações a partir dos dados coletados anteriormente.

In [470]:
def line_off_neighborhoods(m):
  fields = ['name']

  fl.GeoJson(
    neighborhoods.to_json(),
    name='Bairros',
    tooltip=fl.GeoJsonTooltip(labels=False, fields=fields, max_width=100)
  ).add_to(m)


def mark_out_students(m):
  fields = ['student_name', 'name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='blue', icon='home')
  )

  fl.GeoJson(
    G_nodes_student,
    name='Estudantes',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)
  ).add_to(m)


def mark_out_hospitals(m):
  fields = ['name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='red', icon='h-square', prefix='fa')
  )

  fl.GeoJson(
    G_nodes_hospital,
    name='Hospitais',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)
  ).add_to(m)


def mark_out_childcare(m):
  fields = ['name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='green', icon='child', prefix='fa')
  )

  fl.GeoJson(
    G_nodes_daycare,
    name='Creches',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)

  ).add_to(m)


def mark_out_schools(m):
  fields = ['name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='orange', icon='graduation-cap', prefix='fa')
  )

  fl.GeoJson(
    G_nodes_school,
    name='Escolas',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)
  ).add_to(m)


def mark_out_psychosocial_care(m):
  fields = ['name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='purple', icon='user-md', prefix='fa')
  )

  fl.GeoJson(
    G_nodes_psychiatric_care,
    name='Centro de Psicossocial',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)
  ).add_to(m)


def mark_out_points_of_care(m):
  fields = ['name', 'neighborhood_name']

  marker = fl.Marker(
    icon=fl.Icon(color='gray', icon='medkit', prefix='fa')
  )

  fl.GeoJson(
    G_nodes_health_points,
    name='Pontos de Saúde',
    marker=marker,
    tooltip=fl.GeoJsonTooltip(fields=fields, max_width=100)
  ).add_to(m)


def make_map():
  map = fl.Map(location=[G_center.y, G_center.x], zoom_start=14)

  line_off_neighborhoods(map)

  mark_out_students(map)
  mark_out_hospitals(map)
  mark_out_childcare(map)
  mark_out_schools(map)
  mark_out_psychosocial_care(map)
  mark_out_points_of_care(map)

  return map

In [471]:
m = make_map()

fl.LayerControl() \
  .add_to(m)

m

A partir da residência de cada integrante da equipe, quantos serviços públicos estão disponíveis num raio de 1km, 5km e 10km?

In [447]:
services = [
  ESCOLA_TAG,
  CRECHE_TAG,
  HOSPITAL_TAG,
  PONTO_SAUDE_TAG,
  PSICOSSOCIAL_TAG
]

def get_graph_from_node(start_node: int, distance: int) -> nx.MultiDiGraph:
  return ox.truncate.truncate_graph_dist(G, start_node, dist=distance, weight='length')

def get_services_in_graph(graph: nx.MultiDiGraph) -> list[int]:
  services_in_graph = []

  nodes, _ = ox.graph_to_gdfs(graph)

  for idx, node in nodes.iterrows():
    if node['label'] in services:
      services_in_graph.append(idx)

  return services_in_graph

def get_services_nearby(student_name: str, d: int) -> dict[int, list[int]]:
  student_node = get_student_node_id(student_name)

  graph = get_graph_from_node(student_node, d)

  services = get_services_in_graph(graph)

  return services